<a href="https://colab.research.google.com/github/Vishnuvardhan172709/Aiml-/blob/main/assignment%203part.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
print(4.1)
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        for i in range(3):
            print('| ' + ' | '.join(self.board[i * 3:(i + 1) * 3]) + ' |')
            if i < 2:
                print('-------------')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        return ' ' in self.board

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter

    def winner(self, square, letter):
        row_ind = square // 3
        if all([spot == letter for spot in self.board[row_ind * 3:(row_ind + 1) * 3]]):
            return True
        col_ind = square % 3
        if all([self.board[col_ind + i * 3] == letter for i in range(3)]):
            return True
        if square % 2 == 0:
            if all([self.board[i] == letter for i in [0, 4, 8]]) or all([self.board[i] == letter for i in [2, 4, 6]]):
                return True
        return False

    def reset_board(self):
        self.board = [' ' for _ in range(9)]
        self.current_winner = None


class Player:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        raise NotImplementedError


class RandomPlayer(Player):
    def get_move(self, game):
        square = random.choice(game.available_moves())
        return square


class AlphaBetaPlayer(Player):
    def get_move(self, game):
        best_score = float('-inf')
        best_move = None

        for move in game.available_moves():
            game.make_move(move, self.letter)
            score = self.minimax(game, False, float('-inf'), float('inf'))
            game.board[move] = ' '
            if score > best_score:
                best_score = score
                best_move = move

        return best_move

    def minimax(self, game, is_maximizing, alpha, beta):
        if game.current_winner == self.letter:
            return 1
        elif game.current_winner == 'X':
            return -1
        elif not game.empty_squares():
            return 0

        if is_maximizing:
            best_score = float('-inf')
            for move in game.available_moves():
                game.make_move(move, self.letter)
                score = self.minimax(game, False, alpha, beta)
                game.board[move] = ' '
                best_score = max(score, best_score)
                alpha = max(alpha, score)
                if beta <= alpha:
                    break
            return best_score
        else:
            best_score = float('inf')
            for move in game.available_moves():
                game.make_move(move, 'X')
                score = self.minimax(game, True, alpha, beta)
                game.board[move] = ' '
                best_score = min(score, best_score)
                beta = min(beta, score)
                if beta <= alpha:
                    break
            return best_score


def play_game(game, players, verbose=True):
    game.reset_board()
    current_player = 0

    while game.empty_squares() and game.current_winner is None:
        if verbose:
            game.print_board()
            print(f"{players[current_player].letter}'s turn")

        square = players[current_player].get_move(game)
        game.make_move(square, players[current_player].letter)

        if game.current_winner:
            if verbose:
                game.print_board()
                print(f'{players[current_player].letter} wins!')
            return players[current_player].letter

        current_player = (current_player + 1) % 2

    if verbose:
        game.print_board()
        print("It's a tie!")
    return None


tictactoe = TicTacToe()
players = [RandomPlayer('X'), AlphaBetaPlayer('O')]

play_game(tictactoe, players)

4.1
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
X's turn
|   |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
O's turn
| O |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
O wins!


'O'

In [2]:
import random
print(4.2)
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]
        self.current_player = 'X'

    def actions(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def result(self, action):
        new_board = self.board[:]
        new_board[action] = self.current_player
        return new_board

    def is_terminal(self):
        win_conditions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),
            (0, 3, 6), (1, 4, 7), (2, 5, 8),
            (0, 4, 8), (2, 4, 6)
        ]
        for a, b, c in win_conditions:
            if self.board[a] == self.board[b] == self.board[c] != ' ':
                return True
        return all(spot != ' ' for spot in self.board)

    def utility(self):
        """Returns the utility value of the board."""
        win_conditions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),
            (0, 3, 6), (1, 4, 7), (2, 5, 8),
            (0, 4, 8), (2, 4, 6)
        ]
        for a, b, c in win_conditions:
            if self.board[a] == 'X':
                return 1
            elif self.board[a] == 'O':
                return -1
        return 0

    def play(self, player_dict, verbose=False):
        while not self.is_terminal():
            if verbose:
                print(self)
            action = player_dict[self.current_player](self)
            self.board = self.result(action)
            self.current_player = 'O' if self.current_player == 'X' else 'X'

        if verbose:
            print(self)
            if self.utility() == 1:
                print("X wins!")
            elif self.utility() == -1:
                print("O wins!")
            else:
                print("It's a draw!")

    def __str__(self):
        return f"{self.board[0]} | {self.board[1]} | {self.board[2]}\n" \
               f"---------\n" \
               f"{self.board[3]} | {self.board[4]} | {self.board[5]}\n" \
               f"---------\n" \
               f"{self.board[6]} | {self.board[7]} | {self.board[8]}"

def minimax(game):
    if game.is_terminal():
        return game.utility()

    best_value = float('inf')
    for action in game.actions():
        game.board[action] = 'O'
        value = minimax(game)
        game.board[action] = ' '
        best_value = min(best_value, value)

    return best_value

def alphabeta(game, alpha=-float('inf'), beta=float('inf')):
    if game.is_terminal():
        return game.utility()

    best_value = -float('inf')
    for action in game.actions():
        game.board[action] = 'X'
        value = alphabeta(game, alpha, beta)
        game.board[action] = ' '
        best_value = max(best_value, value)
        alpha = max(alpha, value)
        if beta <= alpha:
            break  # Beta cut-off

    return best_value

def player(search_algorithm):
    def make_move(game):
        best_action = None
        best_value = -float('inf') if game.current_player == 'X' else float('inf')

        for action in game.actions():
            game.board[action] = game.current_player
            value = search_algorithm(game)
            game.board[action] = ' '

            if game.current_player == 'X' and value > best_value:
                best_value = value
                best_action = action
            elif game.current_player == 'O' and value < best_value:
                best_value = value
                best_action = action

        return best_action

    return make_move

if __name__ == "__main__":
    players = {
        'X': player(alphabeta),
        'O': player(minimax)
    }
    game_instance = TicTacToe()
    game_instance.play(players, verbose=True)

4.2
  |   |  
---------
  |   |  
---------
  |   |  
X |   |  
---------
  |   |  
---------
  |   |  
X | O |  
---------
  |   |  
---------
  |   |  
X | O | X
---------
  |   |  
---------
  |   |  
X | O | X
---------
O |   |  
---------
  |   |  
X | O | X
---------
O | X |  
---------
  |   |  
X | O | X
---------
O | X | O
---------
  |   |  
X | O | X
---------
O | X | O
---------
X |   |  
X wins!
